In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    recall_score, precision_score, f1_score
)
from imblearn.over_sampling import SMOTE


In [2]:
df = pd.read_csv("creditcard.csv")


In [3]:
df.shape

(284807, 31)

In [4]:
df = df.sample(n = 100000, random_state = 42, replace = False)


In [5]:
X = df.drop(columns=['Class'])
y = df['Class'].astype(int)

In [6]:
# ---- 2) Podział train/test (zachowujemy proporcje klas) ----
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [7]:
oversample = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = oversample.fit_resample(X_train, y_train)


In [8]:
results = []
def evaluation(model, X_train, X_test, y_train, y_test):
    # Predykcje
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print("=" * 50)
    print("CONFUSION MATRIX - TRAIN")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))
    print("Accuracy of train data:", round(accuracy_score(y_train, y_train_pred), 4))
    print("F1-score of train data:", round(f1_score(y_train, y_train_pred, average='macro', zero_division=0), 4))
    print("=" * 50)

    print("CONFUSION MATRIX - TEST")
    print(confusion_matrix(y_test, y_test_pred))
    print(classification_report(y_test, y_test_pred))

    acc = accuracy_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred, average='macro', zero_division=0)
    precision = precision_score(y_test, y_test_pred, zero_division=0)
    recall = recall_score(y_test, y_test_pred, zero_division=0)

    print("Accuracy of test data:", round(acc, 4))
    print("F1-score of test data:", round(f1, 4))
    print("Precision of test data:", round(precision, 4))
    print("Recall of test data:", round(recall, 4))
    print("=" * 50)

    results.append({
    "Model": model,
    "Precision": precision,
    "Recall": recall,
    "Accuracy": acc,
    "F1": f1

    })


    # Zwróć metryki
    return {
        "Model": model.__class__.__name__,
        "Precision": precision,
        "Recall": recall,
        "Accuracy": acc,
        "F1": f1,
    }


In [9]:
# Logistic regression model
logi = LogisticRegression(max_iter=1500, fit_intercept=True)
logi.fit(X_train_resampled, y_train_resampled)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1500
,multi_class,'deprecated'


In [10]:
#class prediction of y
y_pred_logi = logi.predict(X_test)
y_train_pred_logi=logi.predict(X_train_resampled)
evaluation(logi, X_train_resampled, X_test, y_train_resampled, y_test)

CONFUSION MATRIX - TRAIN
[[79194   677]
 [ 2441 77430]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     79871
           1       0.99      0.97      0.98     79871

    accuracy                           0.98    159742
   macro avg       0.98      0.98      0.98    159742
weighted avg       0.98      0.98      0.98    159742

Accuracy of train data: 0.9805
F1-score of train data: 0.9805
CONFUSION MATRIX - TEST
[[19802   166]
 [    1    31]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     19968
           1       0.16      0.97      0.27        32

    accuracy                           0.99     20000
   macro avg       0.58      0.98      0.63     20000
weighted avg       1.00      0.99      0.99     20000

Accuracy of test data: 0.9917
F1-score of test data: 0.6333
Precision of test data: 0.1574
Recall of test data: 0.9688


{'Model': 'LogisticRegression',
 'Precision': 0.15736040609137056,
 'Recall': 0.96875,
 'Accuracy': 0.99165,
 'F1': 0.6332716592887293}

In [11]:
# 3a) K-NN + GridSearch (skaler konieczny)
knn_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

knn_param_grid = {
    "knn__n_neighbors": [3, 5, 7],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2],  # 1=Manhattan, 2=Euclidean
}

knn_grid = GridSearchCV(
    knn_pipe, knn_param_grid,
    scoring="average_precision",  # PR-AUC lepsza przy niezrównoważeniu klas
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1
)
knn_grid.fit(X_train, y_train)
best_knn = knn_grid.best_estimator_

In [12]:
evaluation(best_knn, X_train_resampled, X_test, y_train_resampled, y_test)


CONFUSION MATRIX - TRAIN
[[79871     0]
 [14073 65798]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92     79871
           1       1.00      0.82      0.90     79871

    accuracy                           0.91    159742
   macro avg       0.93      0.91      0.91    159742
weighted avg       0.93      0.91      0.91    159742

Accuracy of train data: 0.9119
F1-score of train data: 0.9112
CONFUSION MATRIX - TEST
[[19965     3]
 [    5    27]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19968
           1       0.90      0.84      0.87        32

    accuracy                           1.00     20000
   macro avg       0.95      0.92      0.94     20000
weighted avg       1.00      1.00      1.00     20000

Accuracy of test data: 0.9996
F1-score of test data: 0.9354
Precision of test data: 0.9
Recall of test data: 0.8438


{'Model': 'Pipeline',
 'Precision': 0.9,
 'Recall': 0.84375,
 'Accuracy': 0.9996,
 'F1': 0.935383715727119}

In [13]:
#fitting data into Decision Tree Classifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train_resampled, y_train_resampled)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [14]:
y_pred_dtc = dtc.predict(X_test)
y_train_pred_dtc=dtc.predict(X_train_resampled)

In [15]:
evaluation(dtc, X_train_resampled, X_test, y_train_resampled, y_test)


CONFUSION MATRIX - TRAIN
[[79871     0]
 [    0 79871]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     79871
           1       1.00      1.00      1.00     79871

    accuracy                           1.00    159742
   macro avg       1.00      1.00      1.00    159742
weighted avg       1.00      1.00      1.00    159742

Accuracy of train data: 1.0
F1-score of train data: 1.0
CONFUSION MATRIX - TEST
[[19922    46]
 [    6    26]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19968
           1       0.36      0.81      0.50        32

    accuracy                           1.00     20000
   macro avg       0.68      0.91      0.75     20000
weighted avg       1.00      1.00      1.00     20000

Accuracy of test data: 0.9974
F1-score of test data: 0.7493
Precision of test data: 0.3611
Recall of test data: 0.8125


{'Model': 'DecisionTreeClassifier',
 'Precision': 0.3611111111111111,
 'Recall': 0.8125,
 'Accuracy': 0.9974,
 'F1': 0.7493483055945458}

In [16]:
results_df = pd.DataFrame(results).sort_values(by="Recall", ascending=False)

In [17]:
round(results_df,3, )

,Model,Precision,Recall,Accuracy,F1
0,LogisticRegression(max_iter=1500),0.157,0.969,0.992,0.633
1,"(StandardScaler(), KNeighborsClassifier(n_neig...",0.900,0.844,1.000,0.935
2,DecisionTreeClassifier(),0.361,0.812,0.997,0.749


In [18]:
# Podsumowanie:
# Model o najwyższej wartości Recall wykrywa największą liczbę rzeczywistych przypadków oszustw (fraudów),
# nawet kosztem większej liczby fałszywych alarmów (False Positives).
# Ponieważ w tym zadaniu priorytetem jest minimalizacja liczby pominiętych fraudów (False Negatives),
# kluczową metryką oceny modeli jest właśnie Recall.
# Wyniki modeli zostały posortowane według tej miary (malejąco),
# dlatego jako model końcowy należy wybrać ten, który znajduje się na pierwszym miejscu w tabeli wyników.
